In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Authenticate from the terminal:

```bash
$ earthengine authenticate --quiet
```

Sign in from the web browser, and paste in the authorization code.
Then proceed ...


In [1]:
import ee
import geopandas as gpd
import json
import pandas as pd
import numpy as np
import geemap
import pathlib

In [2]:
# Initialize the library.
ee.Initialize()

In [3]:
layer = ee.Image("CSP/ERGo/1_0/Global/SRTM_mTPI")
srtmMtpi = layer.select('elevation')

In [4]:
shp = gpd.read_file('/home/forecol/amelia/Amazon_rainforest_shapefile.zip')
js = json.loads(shp.to_json())
geometry = ee.Geometry(ee.FeatureCollection(js).geometry())

In [26]:
srtmMtpi_clipped = srtmMtpi.clip(geometry)
srtm_client = srtmMtpi_clipped.getInfo()

In [ ]:
srtm_df = pd.DataFrame(data=srtm_client[1:], columns=srtm_client[0])
srtm_gdf = gpd.GeoDataFrame(
    data=srtm_df.elevation, 
    geometry=gpd.points_from_xy(srtm_df.longitude, srtm_df.latitude)
)


In [5]:
# 1. turn a dataframe of filtered GEDI shots into an EE object
l4a_data = pd.read_feather('/home/forecol/data/Overlays/GEDI4A_JRC_filtering/dfs_all.feather')
l4a_data['agcd'] = l4a_data.agbd * 0.48
quality_gedi = l4a_data[
    (l4a_data.overlap_quality > 1) &
    (l4a_data.overlap_quality != 3) &
    (l4a_data.recovery_period >= 3) &
    (l4a_data.recovery_period <= 22) &
    (l4a_data.l4_quality_flag == 1) &
    (l4a_data.l2_quality_flag == 1) &
    (l4a_data.degrade_flag == 0) #&
]
print(len(quality_gedi))

quality_gedi_gdf = gpd.GeoDataFrame(
    quality_gedi, 
    geometry=gpd.points_from_xy(
        quality_gedi.lon_lowestmode, quality_gedi.lat_lowestmode),
    crs='EPSG:4326')

js = json.loads(quality_gedi_gdf.to_json())
fc = ee.FeatureCollection(js)


96348


In [6]:
# GEDI shots have a 25m radius, so buffer the locations with a 25 m circle
def bufferPoints(radius: int, bounds: bool):
  return lambda pt: pt.buffer(radius).bounds() if bounds else pt.buffer(radius)

shots = fc.map(bufferPoints(25, False));

In [8]:
# 2. Get raster values for the GEDI shot locations
# See the guide at https://developers.google.com/earth-engine/tutorials/community/extract-raster-values-for-points

def zonalStats(
    ic: ee.ImageCollection, 
    fc: ee.FeatureCollection,
    reducer = ee.Reducer.mean(),
    scale = None,
    crs: str = None,
    bands = None,
    bandsRename = None,
    imgProps = None,
    imgPropsRename = None,
    datetimeName: str = 'datetime',
    datetimeFormat: str = 'YYYY-MM-dd HH:mm:ss'):

    # Set additional default params based on an image representative.
    imgRep = ic.first()
    if not bands:
        bands = imgRep.bandNames()
    if not bandsRename:
        bandsRename = bands
    if not imgProps:
        imgProps = ee.Feature(None).copyProperties(imgRep).propertyNames()
    if not imgPropsRename:
        imgPropsRename = imgProps

    def reduceImage(img: ee.Image):
        # Select bands (optionally rename), set a datetime & timestamp property.
        img = ee.Image(img.select(bands, bandsRename))\
            .set(datetimeName, img.date().format(datetimeFormat))\
            .set('timestamp', img.get('system:time_start'))

        # Define final image properties to set in output features.
        propsFrom = ee.List(imgProps)\
            .cat(ee.List([datetimeName, 'timestamp']))
        propsTo = ee.List(imgPropsRename)\
            .cat(ee.List([datetimeName, 'timestamp']))
        imgProps_new = img.toDictionary(propsFrom).rename(propsFrom, propsTo)

        # subset points that intersect the given image
        fcSub = fc.filterBounds(img.geometry())

        # reduce the image by regions
        return img.reduceRegions(
            collection=fcSub,
            reducer=reducer,
            scale=scale,
            crs=crs
        ).map(lambda f: f.set(imgProps_new))

    # Map the reduceRegions function over the image collection.
    results = ic.map(reduceImage).flatten().filter(ee.Filter.notNull(bandsRename))
    return results
    

srtmMtpi_coll = ee.ImageCollection([srtmMtpi])

pts_srtmMtpiStats = zonalStats(srtmMtpi_coll, shots, bandsRename=['elevation'])

In [17]:
i = pts_srtmMtpiStats.first().getInfo()
print(i)

EEException: Request payload size exceeds the limit: 10485760 bytes.

In [12]:
task = ee.batch.Export.table.toDrive(
    collection=pts_srtmMtpiStats, 
    description='tpi-gedi-overlay',
    fileFormat='CSV',
    selectors=['shot_number', 'elevation']
)

In [13]:
taskdata = task.start()

EEException: Request payload size exceeds the limit: 10485760 bytes.

In [14]:
print(task)


<Task EXPORT_FEATURES: tpi-gedi-overlay (UNSUBMITTED)>


In [15]:
geemap.ee_to_csv(pts_srtmMtpiStats, '/home/forecol/data/Overlays/GEE_layers/elevation.csv')

Request payload size exceeds the limit: 10485760 bytes.


In [ ]:
srtmMtpiStats_df = pd.read_csv('/home/forecol/data/Overlays/GEE_layers/elevation.csv')

In [ ]:
del srtmMtpiStats_client

In [ ]:
# ** GEE only offers a random forest classification algo, not regression
# see https://gis.stackexchange.com/questions/316739/difference-between-gee-and-sklearn-random-forest-output
# for random forest sklearn regression snippet